# Neural & Behavioral Modeling - Week 5 (Exercises)
by 劉昱維 (r08227111@g.ntu.edu.tw)

In [1]:
%config IPCompleter.greedy=True 
%matplotlib inline
from numpy import *
from matplotlib.pyplot import *
from IPython.display import *

## 1 Replicate exisiting findings/models (10 points)
Based on your personal interest, please choose and replicate ONE of the following studies:

1. <a href="http://ccl.northwestern.edu/netlogo/models/FlockingVeeFormations">Flocking Vee Formations</a> in NetLogo's [Sample Models::Biology] 

2. <a href="http://ccl.northwestern.edu/netlogo/models/WolfSheepPredation">Wolf-Sheep Predation</a> in NetLogo's [Sample Models::Biology] 

3. <a href="https://kknews.cc/zh-tw/news/p56pbrp.html">MIT Matching Game</a>

4. Gray, K., Rand, D. G., Ert, E., Lewis, K., Hershman, S., & Norton, M. I. (2014). <a href="http://www.mpmlab.org/groups/">The emergence of “us and them” in 80 lines of code: Modeling group genesis in homogeneous populations</a>. Psychological Science, 25(4), 982-990.

5. Luhmann, C. C., & Rajaram, S. (2015). <a href="http://journals.sagepub.com/doi/abs/10.1177/0956797615605798">Memory transmission in small groups and large networks: An agent-based model</a>. Psychological Science, 26(12), 1909-1917.

6. Henriques, G. J., Simon, B., Ispolatov, Y., & Doebeli, M. (2019). <a href="https://www.pnas.org/content/116/28/14089">Acculturation drives the evolution of intergroup conflict</a>. Proceedings of the National Academy of Sciences, 116(28), 14089-14097.

## Replicate No.1
### 使用vpython 重製 Flocking Vee Formations
參考來自
Eric Nilsen
September 2003
:https://vpython.org/contents/contributed/boids.py<br>
1. 其版本Py2已經過時無法使用，將其修復。<br>
[Module modified] <br>
-- python2 -> python3<br>
-- visual -> vpython<br>
[Function modified]<br>
-- Add getLocate()<br>
-- Fix rule1<br>
-- Fix rule2<br>
-- Fix rule3<br>
-- Fix boids might fly out off box bug <br>


In [ ]:
# Please install vpython first.
!pip install vpython  

In [1]:
import vpython
from vpython import *
from random import randrange

class Boids:
    def __init__(self, numboids = 100, sidesize = 100.0):     
        #class constants 
        display(title = "Boids v1.0")   # 設置標題 

        self.SIDE = sidesize            # 定義飛行空間 100*100*100

        #the next six lines define the boundaries of the torus
        
        
        self.MINX = self.SIDE * -1.0    # 左
        self.MINY = self.SIDE * -1.0    # 下
        self.MINZ = self.SIDE * -1.0    # 後
        self.MAXX = self.SIDE           # 右
        self.MAXY = self.SIDE           # 上
        self.MAXZ = self.SIDE           # 前
        
        self.RADIUS = 3                 # 鳥的大小(用球代替)
        self.NEARBY = self.RADIUS * 5   # 鳥彼此之間的容許距離
        self.FACTOR = .99               # 鳥對容許距離內，覺察中心的比例
        self.NEGFACTOR = self.FACTOR * -1.0 # 同上，但因為是3D handle前後case，再設一個負的
        
        self.NUMBOIDS = numboids        # 鳥數

        self.boidflock = []             
        self.DT = 0.02                  # delay time between snapshots

        self.boids()                    # init

    def boids(self):
        self.initializePositions()      # 創建鳥
        while (1==1):                   
            rate(100)                   # controls the animation speed, bigger = faster
            self.moveAllBoidsToNewPositions()   
    
    def getLocate(self,boid):
        return boid._pos.x, boid._pos.y, boid._pos.z
    
    def initializePositions(self):
        #wire frame of space
        backBottom = curve(pos=[(self.MINX, self.MINY, self.MINZ), (self.MAXX, self.MINY, self.MINZ)], color=color.white)
        backTop = curve(pos=[(self.MINX, self.MAXY, self.MINZ), (self.MAXX, self.MAXY, self.MINZ)], color=color.white)
        frontBottom = curve(pos=[(self.MINX, self.MINY, self.MAXZ), (self.MAXX, self.MINY, self.MAXZ)], color=color.white)
        frontTop = curve(pos=[(self.MINX, self.MAXY, self.MAXZ), (self.MAXX, self.MAXY, self.MAXZ)], color=color.white)
        leftBottom = curve(pos=[(self.MINX, self.MINY, self.MINZ), (self.MINX, self.MINY, self.MAXZ)], color=color.white)
        leftTop = curve(pos=[(self.MINX, self.MAXY, self.MINZ), (self.MINX, self.MAXY, self.MAXZ)], color=color.white)
        rightBottom = curve(pos=[(self.MAXX, self.MINY, self.MINZ), (self.MAXX, self.MINY, self.MAXZ)], color=color.white)
        rightTop = curve(pos=[(self.MAXX, self.MAXY, self.MINZ), (self.MAXX, self.MAXY, self.MAXZ)], color=color.white)
        backLeft = curve(pos=[(self.MINX, self.MINY, self.MINZ), (self.MINX, self.MAXY, self.MINZ)], color=color.white)
        backRight = curve(pos=[(self.MAXX, self.MINY, self.MINZ), (self.MAXX, self.MAXY, self.MINZ)], color=color.white)
        frontLeft = curve(pos=[(self.MINX, self.MINY, self.MAXZ), (self.MINX, self.MAXY, self.MAXZ)], color=color.white)
        frontRight = curve(pos=[(self.MAXX, self.MINY, self.MAXZ), (self.MAXX, self.MAXY, self.MAXZ)], color=color.white)

        #splatter a flock in the space randomly
                                          #initialize the color switch
        for b in range(self.NUMBOIDS):          #for each boid, ...
            x = randrange(self.MINX, self.MAXX) #random left-right
            y = randrange(self.MINY, self.MAXY) #random up-down
            z = randrange(self.MINZ, self.MAXZ) #random front-back

            
            COLOR = color.yellow
            #splat a boid, add to flock list
            self.boidflock.append(sphere(pos=vector(x,y,z), radius=self.RADIUS, color=COLOR))

    def moveAllBoidsToNewPositions(self):
        for b in range(self.NUMBOIDS):
            #manage boids hitting the torus 'boundaries'
            bx,by,bz = self.getLocate(self.boidflock[b])
            if bx == self.MINX or bx < self.MINX:
                self.boidflock[b]._pos.x = self.MINX + 1
                
            if bx > self.MAXX or bx == self.MAXX:
                self.boidflock[b]._pos.x = self.MAXX -1
                
            if by < self.MINY:
                self.boidflock[b]._pos.y = self.MINY +1
                
            if by > self.MAXY:
                self.boidflock[b]._pos.y = self.MAXY -1 
                
            if bz < self.MINZ:
                self.boidflock[b]._pos.z = self.MINZ + 1
                
            if bz > self.MAXZ:
                self.boidflock[b]._pos.z = self.MAXZ - 1
            

            v1 = vector(0.0,0.0,0.0)        #initialize vector for rule 1
            v2 = vector(0.0,0.0,0.0)        #initialize vector for rule 2
            v3 = vector(0.0,0.0,0.0)        #initialize vector for rule 3

 
            v1 = self.rule1(b)              #get the vector for rule 1
            v2 = self.rule2(b)              #get the vector for rule 2
            v3 = self.rule3(b)              #get the vector for rule 3

            boidvelocity = vector(0.0,0.0,0.0)          #initialize the boid velocity
            boidvelocity = boidvelocity + v1 + v2 + v3  #accumulate the rules vector results
            self.boidflock[b].pos = self.boidflock[b].pos + (boidvelocity*self.DT) #move the boid

                

    def rule1(self, aboid):    #Rule 1:  boids fly to perceived flock center
        pfc = vector(0.0,0.0,0.0)                   #pfc: perceived flock center
        for b in range(self.NUMBOIDS):              #for all the boids
            if b != aboid:                          #except the boid at hand
                 pfc = pfc + self.boidflock[b]._pos  #calculate the total pfc

        pfc = pfc/(self.NUMBOIDS - 1.0)             #average the pfc
##        self.greenie.pos = pfc                      #put greenie at the pfc, see what's going on

        #nudge the boid in the correct direction toward the pfc 
        if pfc.x > self.boidflock[aboid]._pos.x:
            pfc.x = (pfc.x - self.boidflock[aboid].pos.x)*self.FACTOR
        if pfc.x < self.boidflock[aboid]._pos.x:
            pfc.x = (self.boidflock[aboid]._pos.x - pfc.x)*self.NEGFACTOR
        if pfc.y > self.boidflock[aboid]._pos.y:
            pfc.y = (pfc.y - self.boidflock[aboid]._pos.y)*self.FACTOR
        if pfc.y < self.boidflock[aboid]._pos.y:
            pfc.y = (self.boidflock[aboid]._pos.y - pfc.y)*self.NEGFACTOR
        if pfc.z > self.boidflock[aboid]._pos.z:
            pfc.z = (pfc.z - self.boidflock[aboid]._pos.z)*self.FACTOR
        if pfc.z < self.boidflock[aboid]._pos.z:
            pfc.z = (self.boidflock[aboid]._pos.z - pfc.z)*self.NEGFACTOR

        return pfc 

    def rule2(self, aboid):    #Rule 2: boids avoid other boids
        v = vector(0.0,0.0,0.0) #initialize the avoidance vector

        for b in range(self.NUMBOIDS):
            if b != aboid:
                bx,by,bz = self.getLocate(self.boidflock[b])
                aboidx, aboidy, aboidz = self.getLocate(self.boidflock[aboid])
                if abs(bx - aboidx) < self.NEARBY:
                    if bx > aboidx:
                        v.x = self.NEARBY * 12.0    #works better when I multiply by 12, don't know why
                    if bx < aboidx:
                        v.x = -self.NEARBY * 12.0
                if abs(by - aboidy) < self.NEARBY:
                    if by > aboidy:
                        v.y = self.NEARBY * 12.0
                    if by < aboidy:
                        v.y = -self.NEARBY * 12.0
                if abs(bz - aboidz) < self.NEARBY:
                    if bz > aboidz:
                        v.z = self.NEARBY * 12.0
                    if bz < aboidz:
                        v.z = -self.NEARBY * 12.0

        return v
        
    def rule3(self, aboid):    #Rule 3: boids try to match speed of flock
        pfv = vector(0.0,0.0,0.0)   #pfv: perceived flock velocity
            
        for b in range(self.NUMBOIDS):
            if b != aboid:
                 pfv = pfv + self.boidflock[b].pos

        pfv = pfv/(self.NUMBOIDS - 1.0)
        pfv = pfv/(aboid + 1)    #some of the boids are more sluggish than others
        
        return pfv


if __name__ == "__main__":
    b = Boids(100,200)     


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

KeyboardInterrupt: 